# EURECOM - Semester Project Fall 2018
## Title: Machine Learning in the Kitchen

-----

# 3 Data cleaning and preparation 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import csv 
import matplotlib.pyplot as plt
%matplotlib inline

## Dataset #1

Dataset #1 contains 5500 recipes, the dimentionality of this dataset (number of ingridients) = 3926.  
The link to the dataset: https://raw.githubusercontent.com/trungduynguyen/EURECOM-semester-project/master/data/

In [ ]:
import urllib.request
import requests

#url_src = 'https://raw.githubusercontent.com/trungduynguyen/EURECOM-semester-project/master/data/'
#path = '/home/vlbackend/EURECOM/data/' 

# download data from github
for i in range(11):
    url = url_src + 'data' + str(i) + '.csv'
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    data = response.read()
    dest_file = path + 'data' + str(i) + '.csv'
    with open(dest_file, 'wb') as f:
        f.write(data)

In [ ]:
# load data
def load_data():
    data_df = []
    for i in range(0,11):
        temp = pd.DataFrame.from_csv('data/data'+str(i)+'.csv')
        data_df.append(temp)
        
    recipe_df = pd.concat(data_df)  
    shuffle_df = shuffle(recipe_df)
    data = np.delete(shuffle_df.as_matrix(),[3926],1)
    #recipe_labels = shuffle_df.as_matrix()[:,-1] #remove rate
    
    # list of ingridients
    ingridients = list(recipe_df.columns)
    ingridients.pop()
  
    return data,ingridients

data,ingridients = load_data()

In [ ]:
# get information about the dataset
def get_frequency(matrix,ax):
    freq = []
    # count number of ingridients for each recipe
    if ax == 'row':
        for i in range(matrix.shape[0]):
            freq.append(np.count_nonzero(matrix[i,:]))
    # count number of recipes for each ingridient
    if ax == 'column':
        for j in range(matrix.shape[1]):
            freq.append(np.count_nonzero(matrix[:,j]))
    return freq

In [ ]:
number_ingridients_per_recipe = get_frequency(data, ax='row')

plt.figure(figsize=(3,3))
plt.title('histogram:'+'\n'+'number of ingridients per recipes',fontsize=8)
plt.grid(True)
plt.hist(number_ingridients_per_recipe,bins=40,normed=True,color='salmon',alpha=0.8)
plt.show()

print('mean:',np.count_nonzero(data)/5500)
print('max:',np.max(data),'min:',np.min(data))

In [ ]:
number_recipes_per_ingridient = get_frequency(data, ax='column')

for i in range(10):
    colum_ind = np.where(np.array(number_recipes_per_ingridient) == i)
    print(len(colum_ind[0]), 'ingridients take part in', i, 'recipes')

2626 ingridients don't take part in any represented recipes, so we delete them from the dataset, and thus significatly reduce the dimentionality of the dataset.  
  
Since we don't know units of measure of the ingridients in our dataset and users write recipes themselfes using different units, we can convert our real dataset to binary indicationg only the absence or presence of ingridients.

In [ ]:
# delete ingridients
colum_ind_zero = np.where(np.array(number_recipes_per_ingridient) == 0)
cleaned_data = np.delete(data, colum_ind_zero[0], 1)
cleaned_ingridients = np.delete(ingridients, colum_ind_zero[0])

# convert dataset to binary
cleaned_data = (cleaned_data > 0).astype(int)

In [ ]:
for x in range(1300):
    print(x,dict_ingridients[x],"---",cleaned_ingridients[x])

In [ ]:
# save cleaned data
np.savetxt("cleaned_data.csv", cleaned_data, delimiter=",")

# save cleaned ingridients
csvfile = "cleaned_ingridients.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in cleaned_ingridients:
        writer.writerow([val]) 

### Can we clean the data more?

In [ ]:
import csv

def import_data(file,num_rec=5500,num_ingr=1300):
    data = np.zeros((num_rec,num_ingr))
    with open(file) as f:
        reader = csv.reader(f, delimiter=',')
        i = 0
        for row in reader:
            data[i,:] = row
            i += 1
    return data

# import cleaned data
data_file ='cleaned_data.csv'
cleaned_data = import_data(data_file)

In [ ]:
# import list of cleaned ingridients
ingridients_file ='cleaned_ingridients.csv'

with open(ingridients_file) as f:
    reader = csv.reader(f,delimiter=',')
    ingridients = list(reader)
cleaned_ingridients = np.squeeze(ingridients)

# create dictionary of ingridients
dict_ingridients = dict(zip(cleaned_ingridients,range(len(cleaned_ingridients))))

In [ ]:
import re

def clean_dataset(regex, patterns, cleaned_ingridients, cleaned_data, dict_ingridients):
    list_to_delete = []
    for x in range(len(cleaned_ingridients)):
        if re.search(regex, cleaned_ingridients[x]):
            print(x,cleaned_ingridients[x])
            for i in range(len(patterns)):
                cleaned_ingridients[x] = re.sub(patterns[i],"",cleaned_ingridients[x],flags=0)
            print(x,cleaned_ingridients[x])
            num = dict_ingridients.get(cleaned_ingridients[x],-1)
            print('--->',num, cleaned_ingridients[num],"\n")
            if num != -1:
                cleaned_data[:,num] = cleaned_data[:,num] + cleaned_data[:,x]
                list_to_delete.append(x)  

    # delete ingridients
    cleaned_data = np.delete(cleaned_data, list_to_delete, 1)
    cleaned_ingridients = np.delete(cleaned_ingridients, list_to_delete)
    print(len(cleaned_ingridients))
    print(cleaned_data.shape)
    
    # create dictionary of ingridients
    dict_ingridients = dict(zip(cleaned_ingridients,range(len(cleaned_ingridients))))
    return cleaned_ingridients, cleaned_data, dict_ingridients
    
    
regex = r"to taste"
patterns = ["(\sor){0,1}\sto taste\s{0,1}", "^to taste\s"]
cleaned_ingridients,cleaned_data,dict_ingridients = clean_dataset(regex,patterns,cleaned_ingridients,cleaned_data,dict_ingridients)

In [ ]:
regex = r"or more"
patterns = ["\s(or)\s(more)"]
cleaned_ingridients,cleaned_data,dict_ingridients = clean_dataset(regex,patterns,cleaned_ingridients,cleaned_data,dict_ingridients)

In [ ]:
regex = r"^boneless"
patterns = ["^boneless\s"]
cleaned_ingridients,cleaned_data,dict_ingridients = clean_dataset(regex,patterns,cleaned_ingridients,cleaned_data,dict_ingridients)

In [ ]:
regex = r"^baby"
patterns = ["^baby\s"]
cleaned_ingridients,cleaned_data,dict_ingridients = clean_dataset(regex,patterns,cleaned_ingridients,cleaned_data,dict_ingridients)

In [ ]:
regex = r"^whole\s"
patterns = ["^whole\s"]
cleaned_ingridients,cleaned_data,dict_ingridients = clean_dataset(regex,patterns,cleaned_ingridients,cleaned_data,dict_ingridients)

In [ ]:
regex = r"^fresh.*"
patterns = ["^fresh\s"]
cleaned_ingridients,cleaned_data,dict_ingridients = clean_dataset(regex,patterns,cleaned_ingridients,cleaned_data,dict_ingridients)

In [ ]:
regex = r".*concentrate?"
patterns = ["\sconcentrate?"]
cleaned_ingridients,cleaned_data,dict_ingridients = clean_dataset(regex,patterns,cleaned_ingridients,cleaned_data,dict_ingridients)

In [ ]:
np.count_nonzero(cleaned_data)

In [ ]:
# convert dataset to binary
cleaned_data = (cleaned_data > 0).astype(int)

We obtain dataset with 1215 ingridients!

In [ ]:
# save cleaned data
np.savetxt("new_cleaned_data.csv", cleaned_data, delimiter=",")

# save cleaned ingridients
csvfile = "new_cleaned_ingridients.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in cleaned_ingridients:
        writer.writerow([val]) 

## Dataset #2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import csv

In [ ]:
def import_data(file,num_rec=4300,num_ingr=1526):
    data = np.zeros((num_rec,num_ingr))
    with open(file) as f:
        reader = csv.reader(f, delimiter=',')
        i = 0
        for row in reader:
            data[i,:] = row
            i += 1
    return data

# import data
data_file ='SemesterProject-master/data/data.csv'
#data = pd.DataFrame.from_csv(data_file)
data = import_data(data_file)

In [ ]:
np.count_nonzero(data)/4300